In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
cd '/content/drive/MyDrive/Ali Sobhani Thesis/TS'

/content/drive/MyDrive/Ali Sobhani Thesis/TS


In [11]:
!pip install scikit-optimize
!pip install scikeras


In [12]:
from tensorflow.keras.layers import Flatten, Conv2D, DepthwiseConv2D, BatchNormalization, Dense, Activation, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.constraints import max_norm
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV
import numpy as np

X = np.load("X.npy", allow_pickle=True)
Y = np.load("Y.npy")

def create_model1(optimizer='adam', kernel_size=3, dropout=0.5, norm_rate=0.01, F1=16, D=2, activation='relu'):
    input1       = Input(shape = (8, 7, 1))
    block1       = Conv2D(F1, (1, kernel_size), padding = 'same',
                                    input_shape = (1, 8, 7),
                                    use_bias = False)(input1)
    block1       = BatchNormalization(axis = -1)(block1)
    block2       = DepthwiseConv2D((8, 1), use_bias = False,
                                      depth_multiplier = D,
                                      depthwise_constraint = max_norm(norm_rate))(block1)
    block2       = BatchNormalization(axis = -1)(block2)
    if activation == 'leakyrelu':
        block2   = LeakyReLU()(block2)
    else:
        block2   = Activation(activation)(block2)
    flatten      = Flatten(name = 'flatten')(block2)
    block3       = Dropout(dropout)(flatten)
    dense        = Dense(1, name = 'dense',
                          kernel_constraint = max_norm(norm_rate))(block3)
    sigmoid      = Activation('sigmoid', name = 'sigmoid')(dense)
    model        = Model(inputs=input1, outputs=sigmoid)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

p = {
    'optimizer': ['Adam', 'Adamax', 'Nadam'],
    'batch_size': [32, 64, 128],
    'epochs': [10, 15, 20, 30],
    'kernel_size': [2, 3, 4, 5],
    'dropout': [0.2, 0.3, 0.4, 0.5],
    'norm_rate': [0.01, 0.001, 0.1, 1, 5, 10],
    'F1': [16, 32, 64],
    'D': [1, 2, 3],
    'activation': ['relu', 'elu', 'leakyrelu']
}

model = KerasClassifier(model=create_model1, verbose=1, **p)
inner_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=13)
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=13)

bayes = BayesSearchCV(estimator=model, search_spaces=p, n_iter=10, n_points=2, cv=inner_cv, scoring='accuracy')

best_scores = {'accuracy': [], 'f2': [], 'f1':[]}
best_params = []

for train_idx, test_idx in outer_cv.split(X, Y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    bayes_result = bayes.fit(X_train, y_train)

    best_params.append(bayes_result.best_params_)
    #best_scores['accuracy'].append(bayes_result.best_score_)
    y_pred = bayes.predict(X_test)
    f2 = fbeta_score(y_test, y_pred, beta=2)
    ################################
    from sklearn.metrics import f1_score, accuracy_score
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    ################################
    best_scores['f2'].append(f2)
    best_scores['f1'].append(f1)
    best_scores['accuracy'].append(accuracy)
average_best_scores = {scoring: np.mean(scores) for scoring, scores in best_scores.items()}
print(f"Average best scores: {average_best_scores}")


Epoch 1/10
9/9 [==============================] - 2s 6ms/step - loss: 0.7522 - accuracy: 0.5510
Epoch 2/10
9/9 [==============================] - 0s 6ms/step - loss: 0.5239 - accuracy: 0.7538
Epoch 3/10
9/9 [==============================] - 0s 7ms/step - loss: 0.4516 - accuracy: 0.8010
Epoch 4/10
9/9 [==============================] - 0s 10ms/step - loss: 0.3883 - accuracy: 0.8385
Epoch 5/10
9/9 [==============================] - 0s 8ms/step - loss: 0.3384 - accuracy: 0.8635
Epoch 6/10
9/9 [==============================] - 0s 9ms/step - loss: 0.3009 - accuracy: 0.8952
Epoch 7/10
9/9 [==============================] - 0s 9ms/step - loss: 0.2891 - accuracy: 0.8933
Epoch 8/10
9/9 [==============================] - 0s 7ms/step - loss: 0.2666 - accuracy: 0.8990
Epoch 9/10
9/9 [==============================] - 0s 9ms/step - loss: 0.2509 - accuracy: 0.9067
Epoch 10/10
9/9 [==============================] - 0s 3ms/step
Epoch 1/10
9/9 [==============================] - 6s 10ms/step - loss: 0

In [13]:
import pandas as pd
import openpyxl
model = ['Model1', 'Model2', 'Model3', 'Model4']
FS = ['Bayes', 'Random', 'Grid']
# Load the existing file
book = openpyxl.load_workbook('TS_Results.xlsx')

# Prepare the data to be written
data_acc = average_best_scores['accuracy']
data_f2 = average_best_scores['f2']
data_f1 = average_best_scores['f1']

# Get the existing sheets
sheet_acc = book['ACC']
sheet_f2 = book['F2']
sheet_f1 = book['F1']

# Calculate the correct row and column numbers
row = model.index('Model1') + 2  # +2 because Excel index starts from 1 and row 1 contains headers
col = FS.index('Bayes') + 2  # +2 because Excel index starts from 1 and column 1 contains headers

# Write to the ACC sheet
sheet_acc.cell(row=row, column=col, value=data_acc)

# Write to the F2 sheet
sheet_f2.cell(row=row, column=col, value=data_f2)

# Write to the F1 sheet
sheet_f1.cell(row=row, column=col, value=data_f1)

# Save and close the Excel file
book.save('TS_Results.xlsx')

In [14]:
import os
import pickle

# Specify the directory path
dir_path = '/content/drive/MyDrive/Ali Sobhani Thesis/TS/Bayes/Model1'

# Save best_params and all_selected_names to the directory
with open(os.path.join(dir_path, 'best_params.pkl'), 'wb') as f:
    pickle.dump(best_params, f)